# Extending SoS

SoS can be easily extended with new targets, actions, file previewers, and ability to exchange data between SoS and new languages. To make the extension available to other users, you can either create and distribute a separate package, or extend SoS and send us a pull request.

## Understanding `entry_points`

SoS makes extensive use of **entry points**, which allows external modules to register their features in the file system to make them available to other modules. It can be confusing initially but [this stack overflow ticket](http://stackoverflow.com/questions/774824/explain-python-entry-points) explains it quite well and one of the users went ahead and created [a project](https://github.com/RichardBronosky/entrypoint_demo demonstrates) to demonstrate how this feature works.

To register additional feature with SoS, you either need to extend `entry_points` of the `setup.py` of SoS, or create your own package with these `entry_points`. Option `extras_require` can be used to specify additional dependent packages for these features. For example, you can create a package with the following entry_points to provide support for ruby.

```
    entry_points='''
[sos-language]
ruby = sos-ruby.kernel:sos_ruby

[sos-actions]
ruby = sos-ruby.actions:ruby
'''
```

With the installation of this package, `sos` would be able to obtain a class `sos_ruby` from module `sos-ruby.kernel`, and use it to work with the `ruby` language.

## Additional actions and targets

An action is a normal Python that is decorated as `SoS_Action` so you can define any action as follows:

```
from sos.actions import SoS_Action

@SoS_Action(run_mode='run')
def my_action(*args, **kwargs):
    pass
```

You then need to add an entry to `entry_points` as

```
[sos-actions]
my_action = mypackage.mymodule:my_action
```

Adding additional target is similar with a class derived from `BaseTarget`

```
from sos.target import BaseTarget

class my_target(BaseTarget):
    def __init__(self...)

```

You will need to define several member functions for this class, most notably `exists` that checks the existence of the target. The details of this class can be found at the source code of `BaseTarget`. 

After you defined your target, you can make it available to SOS by adding an appropriate entry point

```
[sos-targets]
my_target = mypackage.mymodule:my_target
```

## File format conversion

To convert between sos and another file format, you would need to define a function that accepts source file, destination file, and additional arguments, such as

```
def my_converter(source_file, dest_file, additional_args):
    # parse additional_args to obtain converter-specific options
    # then convert from source_file to dest_file

```

and register the converter in `setup.py` as

```
[sos-converters]
fromExt-toExt: mypackage.mymodule:my_converter
```

Here `fromExt` is file extension without leading dot, `toExt` is destination file extension without leading dot. If `dest_file` is unspecified, the output should be written to standard output.

## Preview additional formats

Adding a preview function is very simple. All you need to is to define a function that returns preview information, and add an entry point to link the function to certain file format.

More specifically, a previewer should be specified as

```
pattern,priority = preview_module:func
```

or

```
module:func,priority = preview_module:func
```

where

1. `pattern` is a pattern that matches incoming filename (see module fnmatch.fnmatch for details)
2. `module:func` specifies a function in module that detects the type of input file.
3. `priority` is an integer number that indicates the priority of previewer in case multiple pattern or function matches the  same file. Developers of third-party previewer can override an existing previewer by specifying a higher priority number.
4. `preview_module:func` points to a function in a module. The function should accept a filename as the only parameter, and  returns either

   a) A string that will be displayed as plain text to standard output.
   b) A dictionary that will be returned as `data` field of `display_data` (see [Jupyter documentation](http://jupyter-client.readthedocs.io/en/latest/messaging.html) for details). The dictionary typically has `text/html` for HTML output, "text/plain" for plain text, and "text/png" for image presentation of the file.

## Support for a language.

SoS needs to know a few things before it can support a language properly,

1. The Jupyter kernel this language uses to work with Jupyer, which is a `ir` kernel for language `R`.
2. How to translate a Python object to a **similar** object in this language
3. How to translate an object in this language to a **similar** object in Python.

It is important to understand that, instead of providing object that is native to the **sender** language (e.g. use [`rpy2`](http://rpy2.bitbucket.org/) to wrap R objects in Python), SoS tries to provide object that is native to the **recipient** language. That is to say, although objects

```
a = 1
b = c(1, 2)
```

are of the same type `numeric` in R, they are translated to Python in different types

```
a = 1
b = [1, 2]
```

To support a new language, you will need to write a Python package that defines a class, say `mylanguage`, with its object providing the following attributes:

1. `kernel_name`: name of the kernel the language uses
2. `init_statements`: a statement that will be executed by the sub-kernel when the kernel starts. This statement usually defines functions to convert object to Python.
3. `repr_of_py_obj`: a function that represents a Python object in the language format. It accepts a name and a Python object, and should return a name and a string. The string will be executed by the **sub-kernel** to construct the object in subkernel.
4. `py_repr_of_obj`: a function that represents one or more variables in the subkernel as a Python dictionary. It accepts a list of names in the subkernel and returns an updated list of names, and a statement in the subkernel language. 
5. `py_from_repr_of_obj`: a function that creates Python objects from the python expression returned by the subkernel. 

The last three functions are used to transfer variables between SoS and the subkernel.  For example, to send a Python object `b = [1, 2]` to `R` (magic `%get`), SoS will

1. call `repr_of_py_obj('b', b)`
2. retrieve return value `('b', 'b <- c(1, 2)')`
3. execute the returned expression in the subkernel to create variable `b` in it.

Note that the function `repr_of_py_obj` can change the variable name because a valid variable name in Python might not be a valid variable name in another language. The function should return the new name to let SoS know that an object with a different name is created in the subkernel. SoS will execute multiple statements altogether if multiple variables are passed.

Sending variables from subkernel (magic `%put`) follows the same idea. Basically, the subkernel returns a string that would be evaluated by SoS to get the variable. This is achieved with help from two functions. For example, to send a `R` object `b <- c(1, 2)` from subkernel `R` to `SoS` (magic `%put`), SoS will

1. call `py_repr_of_obj(['b'])` to obtain a statement, which in this case is something like `py.repr(list(b=b))` where `py.repr` is defined in `init_statements` of the subkernel
2. execute the returned statement in the subkernel
3. retrieve return value from the subkernel, which can be something like `[1] "{'b': [1, 2]}"`. As you can see, because SoS has no control over how the subkernel returns a string, some further processing is needed.
4. call `py_from_repr_of_obj` to process the returned value to get a dictionary `{'b': [1, 2]}`, which will then be merged to the SoS dictionary.

Note that although SoS uses strings to exchange information between sos and subkernels, you do not have to send all information using strings. For example, the `sos.R.kernel` module exchange Python DataFrame and R data.frame using disk files.

## Adding a subcommad (addon)

If you would like to add a complete subcommand as an addon to SoS, you will need to define two functions and add them to `setup.py` as two entry points, one with suffix `:args` and one with suffix `:func`.

```
[sos_addons]
myaddon:args = yourpackage.module:addon_parser
myaddon:func = yourpackage.module:addon_func
```

The `addon_parser` function should use module `argparse` to return an `ArgumentParser` object. SoS would obtain this parser and add it as a subparse of the SoS main parser so that the options can be parsed as

```
sos myaddon options
```

The `addon_func` should be defined as

```
def addon_func(args, unknown_args)
```

with `args` being the parsed known arguments, and `unknown_args` being a list of unknown arguments that you can process by youself.